In [17]:
import xml.etree.ElementTree as ET 
import os
from Preprocessing import Preprocessing
import numpy as np

In [18]:
#Path to input file, before trimming
inputFilename = 'wbg-fal10.xml'

#Constraints required, SameAttendees are converted to NotOverlap
hards = ['NotOverlap', 'SameAttendees']
softs = ['DifferentTime', 'DifferentDays']

In [19]:
#Trims the input file and outputs the trimmed filename
outputfile = Preprocessing.getTrimmedFile(inputFilename, hards, softs)

In [20]:
#Extracts the data from trimmed file
courses, subparts, classes, hardConstraints, softConstraints, students = Preprocessing.extractData(outputfile)

150 Classes extracted
35 Subparts extracted
21 Courses extracted
56 Hard Constraints extracted
0 Soft Constraints extracted
19 Students extracted


In [21]:
import random
# A class's 'selected' attribute index of the currently selected timing, which is initialized as -1
# This loop assigns a random index to 'selected'. This index is always less than the number of timings available with each class
for classID in classes.keys():
    classes[classID].selected = random.randrange(len(classes[classID].availTimes))

In [23]:
#function to check if 2 timings are overlapping
def isOverlapped(timing1, timing2):
    #getting start and end times for both timings
    t1start = int(timing1.start)
    t2start = int(timing2.start)
    t1end = t1start + int(timing1.length)
    t2end = t2start + int(timing2.length)
    #Count of overlapping days
    count = 0

    
    #number of 5-min timeslots overlapped
    dailyOverlap = 0

    #this if is to check if the time of the classes are overlapping. 
    #classes will overlap only if the time of the class overlap on any day
    if (t1end >= t2start) or (t2end >= t1start):
        #interate through all days of the semester
        for i in range(0, timing1.tdays_np.size):
            #Checks if class occur on the same day at any day of the semester
            if (timing1.tdays_np[i] + timing2.tdays_np[i]) == 2:
                #Add one to days overlapped
                count = count + 1
        #Calculates the daily overlap if classes occur at any same day
        if(count>0):
            if(t1end >= t2start):
                dailyOverlap = t1end - t2start + 1
            elif(t2end >= t1start):
                dailyOverlap = t2end - t1start + 1

    #Total 5-min timeslots overlapped in a semester
    violations = count*dailyOverlap

    return violations, dailyOverlap, count

In [24]:
#Check for two timings for DifferentTime constraints
def getSameTimeslots(timing1, timing2):
    #getting start and end times for both timings
    t1start = int(timing1.start)
    t2start = int(timing2.start)
    t1end = t1start + int(timing1.length)
    t2end = t2start + int(timing2.length)

    #Total overlap in time regardless of day or week
    overlapLength = 0

    #Calculates the daily overlap if classes occur at any same day
    if(t1end >= t2start):
        overlapLength = t1end - t2start + 1
    elif(t2end >= t1start):
        overlapLength = t2end - t1start + 1

    #Returns the overlap length
    return overlapLength

In [25]:
#Check for timings for DifferentDay timings 
def getSameDays(timing1, timing2):
    #getting days data from both timings
    t1d = list(timing1.days)
    t2d = list(timing2.days)

    #Count for days overlapped
    count = 0
    #Iterate through all days of the the week
    for i in range(0, 7):
        #Checks if class occur on the same day at any day of the semester
        if (t1d[i] == t1d[i]):
            #Add one to days overlapped
            count = count + 1

    #return number of days the classes occur on the same day
    return count

In [26]:
from itertools import combinations

#Total violations for all constraints
totalHardViolations = 0

#Iterating all hard constraints
for hard in hardConstraints:
    #Check if type is 'NotOverlap'
    if hard.type == 'NotOverlap':

        #Getting list of classes for the corresponding constraint
        classList = hard.classes
        
        #Getting all possible pairs from the list
        pairs = list(combinations(classList, 2)) 
        
        #Iterating through all the pairs
        for pair in pairs:
            #Unpacking thepair
            class1ID, class2ID = pair
            #Extracting class timings
            
            class1Timing = classes[class1ID].availTimes[classes[class1ID].selected]
            
            class2Timing = classes[class2ID].availTimes[classes[class2ID].selected]


            #Getting 'NonOverlap' violations
            violations, dailyOverlap, count = isOverlapped(class1Timing, class2Timing)
            
            #Adding to total violations
            totalHardViolations = totalHardViolations + violations

print(totalHardViolations)
        
            

        

71104


In [27]:
#Total violations for all constraints
totalSoftPenalty = 0

#Iterating all hard constraints
for soft in softConstraints:
    #Getting list of classes for the corresponding constraint
    classList = soft.classes
        
    #Getting all possible pairs from the list
    pairs = list(combinations(classList, 2)) 
        
    constraintTotal = 0
    #Iterating through all the pairs
    for pair in pairs:
        #Unpacking thepair
        class1ID, class2ID = pair
        #Extracting class timings
        class1Timing = classes[class1ID].availTimes[classes[class1ID].selected]
        class2Timing = classes[class2ID].availTimes[classes[class2ID].selected]

        #Check for 'DifferentTime'
        if(soft.type == 'DifferentTime'):
            overlaps = getSameTimeslots(class1Timing, class2Timing)
            constraintTotal = constraintTotal + overlaps
        
        #Check for 'DifferentDays'
        elif(soft.type == 'DifferentDays'):
            count = getSameDays(class1Timing, class2Timing)
            constraintTotal = constraintTotal + count

    #Adding to total soft penalty after multiplying with penalty corresponding to that soft constraint
    totalSoftPenalty = constraintTotal*int(soft.penalty)
        

print(totalSoftPenalty)

0


In [28]:
#Timing penalty calculation
totalTimingPenalty = 0

#Iterating through all the classes
for classID in classes.keys():
    #Getting selected timing from each class
    selectedTiming = classes[classID].availTimes[classes[classID].selected]
    #Adding penalty of selected timing to total timings penalty
    totalTimingPenalty = totalTimingPenalty + int(selectedTiming.penalty)

print(totalTimingPenalty)

208
